In [8]:
import csv
import numpy as np

history = 5
workout_embeddings = ["Nothing"]


def get_data(filename):

    workout_data = []

    with open(filename, "r") as file:
        reader = csv.reader(file)
        current_time = None
        for row in reader:
            if (len(row) < 3): 
                continue
            if row[0] == "Date":
                continue
            if current_time is None:
                current_time = row[0]
                current_workout = []
            if row[0] != current_time:
                workout = [0] * (history - len(current_workout))
                workout.extend(current_workout)


                workout_data.append(workout)
                current_time = row[0]
                current_workout = []
                workout= []
            workout_name = row[3]
            if workout_name not in workout_embeddings:
                workout_embeddings.append(workout_name)
            embed = workout_embeddings.index(workout_name)
            
            if (len(current_workout) > 0 and current_workout[len(current_workout)-1] == embed):
                continue
            current_workout.append(embed)
    X_train = []
    Y_train = []

    for i in range(len(workout_data)):
        workout = workout_data[i]
        length = len(workout)

        for w in range(2,history):

            for j in range(0,length-w):
                zeros = ([0] * (history-w))
                zeros.extend(workout[j:j+w])
                X_train.append(zeros)
                Y_train.append(workout[j+w])


    X_train = np.array(X_train)
    Y_train = np.array(Y_train)

    X_train = np.eye(len(workout_embeddings))[X_train] 
    Y_train = np.eye(len(workout_embeddings))[Y_train]

    return (X_train, Y_train)

(X_train,Y_train) = get_data("strong_training.csv")
(X_test,Y_test) = get_data("strong_verification.csv")






/tmp/ipykernel_332/2807737693.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(X_train)
/tmp/ipykernel_332/2807737693.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Y_train = np.array(Y_train)


IndexError: arrays used as indices must be of integer (or boolean) type

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
3

print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))



# Define the LSTM model
model = Sequential([
    LSTM(units=50, input_shape=(history, len(workout_embeddings)), return_sequences=False),
    Dense(64, activation='relu'),
    Dense(len(workout_embeddings), activation='softmax')

])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=1000, batch_size=16)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test loss: {loss:.3f}, Test accuracy: {accuracy:.3f}')


Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Epoch 1/1000
247/247 [==============================] - 1s 2ms/step - loss: 4.5829 - accuracy: 0.0516
Epoch 2/1000
247/247 [==============================] - 0s 2ms/step - loss: 4.0022 - accuracy: 0.1081
Epoch 3/1000
247/247 [==============================] - 0s 2ms/step - loss: 3.5397 - accuracy: 0.1739
Epoch 4/1000
247/247 [==============================] - 0s 2ms/step - loss: 3.1818 - accuracy: 0.2237
Epoch 5/1000
247/247 [==============================] - 0s 2ms/step - loss: 2.9046 - accuracy: 0.2766
Epoch 6/1000
247/247 [==============================] - 0s 2ms/step - loss: 2.6796 - accuracy: 0.3217
Epoch 7/1000
247/247 [==============================] - 0s 2ms/step - loss: 2.4819 - accuracy: 0.3452
Epoch 8/1000
247/247 [==============================] - 0s 2ms/step - loss: 2.2837 - accuracy: 0.3883
Epoch 9/1000
247/247 [==============================] - 0s 2ms/step - loss: 2.1206 - accuracy: 0

In [7]:
## Save Model
import tensorflow as tf
model.save("workout_model.h5")
print("Model saved as workout_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model=model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter._experimental_lower_tensor_list_ops = False
converter.conversion_print_before_pass = 'all'

converter.inference_output_type = tf.float32

tfmodel = converter.convert()

# Save TFLite model into a .tflite file

open("workout_model.tflite","wb").write(tfmodel)
#Save workout embeddings to file as json
import json
with open("workout_embeddings.json", "w") as file:
    file.write(json.dumps(workout_embeddings))

Model saved as workout_model.h5


INFO:tensorflow:Assets written to: /tmp/tmpy0_ez3pl/assets


INFO:tensorflow:Assets written to: /tmp/tmpy0_ez3pl/assets
2024-06-04 16:37:03.640342: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-06-04 16:37:03.640354: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-06-04 16:37:03.640709: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpy0_ez3pl
2024-06-04 16:37:03.644373: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2024-06-04 16:37:03.644383: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpy0_ez3pl
2024-06-04 16:37:03.683991: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2024-06-04 16:37:03.686445: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2024-06-04 16:37:03.728607: I tensorflow/cc/saved_model/loader.cc:212] Running initializatio

ConverterError: /usr/local/lib/python3.8/dist-packages/tensorflow/python/saved_model/save.py:1325:0: error: 'tf.TensorListReserve' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/usr/local/lib/python3.8/dist-packages/tensorflow/python/saved_model/save.py:1325:0: note: Error code: ERROR_NEEDS_FLEX_OPS
/usr/local/lib/python3.8/dist-packages/tensorflow/python/saved_model/save.py:1325:0: error: 'tf.TensorListFromTensor' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/usr/local/lib/python3.8/dist-packages/tensorflow/python/saved_model/save.py:1325:0: note: Error code: ERROR_NEEDS_FLEX_OPS
/usr/local/lib/python3.8/dist-packages/tensorflow/python/saved_model/save.py:1325:0: error: 'tf.TensorListStack' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitio

In [3]:
# Function to predict next exercise given previous exercises
def predict_next_exercise(previous_exercises):
    # Convert previous exercises to one-hot encoded vector
    input_data = previous_exercises
    input_data = np.eye(len(workout_embeddings))[input_data]
    
    # Make prediction
    predicted_index = np.argmax(model.predict(np.array([input_data]))[0])
    # Convert predicted index to exercise name
    predicted_exercise = workout_embeddings[predicted_index]
    
    return predicted_exercise

# Test the model with some previous exercises
arms_chest = [0,0,0,0,51] # arms but migrates to chest 
tricp_chest = [0,0,0,53,3] # tricep & chest exercise
chest = [0,0,0,44,53] # chest  exercise
legs = [0,0,0,27,25] # all legs
legs_w_arms = [0,0,5,11,25] # confused never done legs with arms so keeps doing legs



# Generate workout of 5 exercises
def generate_workout(previous_exercises):
    print("___________")
    for (i, exercise) in enumerate(previous_exercises):
        print("INPUT Exercise", i+1, ":", workout_embeddings[exercise])
    workout = []
    for _ in range(5):
        predicted_exercise = predict_next_exercise(previous_exercises)
        workout.append(predicted_exercise)
        previous_exercises = previous_exercises[1:] + [workout_embeddings.index(predicted_exercise)]
    print("Generated workout:", workout)
    for (i, exercise) in enumerate(previous_exercises):
        print("Exercise", i+1, ":", workout_embeddings[exercise])
        
# Generate workout of 5 exercises
def generate_nexworkout(previous_exercises):
    print("___________")
    for (i, exercise) in enumerate(previous_exercises):
        print("INPUT Exercise", i+1, ":", workout_embeddings[exercise])
    workout = []
    for _ in range(5):
        predicted_exercise = predict_next_exercise(previous_exercises)
        workout.append(predicted_exercise)
        previous_exercises = previous_exercises[1:] + [workout_embeddings.index(predicted_exercise)]
    print("Generated workout:", workout)
    for (i, exercise) in enumerate(previous_exercises):
        print("Exercise", i+1, ":", workout_embeddings[exercise])
    

generate_workout(arms_chest)
generate_workout(tricp_chest)
generate_workout(chest)
generate_workout(legs)
generate_workout(legs_w_arms)


___________
INPUT Exercise 1 : Nothing
INPUT Exercise 2 : Nothing
INPUT Exercise 3 : Nothing
INPUT Exercise 4 : Nothing
INPUT Exercise 5 : Lat Pulldown (Cable)
1/1 [==============================] - 0s 7ms/step
Generated workout: ['Bicep Curl (Dumbbell)', 'Hammer Curl (Dumbbell)', 'Seated Row (Cable)', 'Bicep Curl (Cable)', 'Triceps Extension']
Exercise 1 : Bicep Curl (Dumbbell)
Exercise 2 : Hammer Curl (Dumbbell)
Exercise 3 : Seated Row (Cable)
Exercise 4 : Bicep Curl (Cable)
Exercise 5 : Triceps Extension
___________
INPUT Exercise 1 : Nothing
INPUT Exercise 2 : Nothing
INPUT Exercise 3 : Nothing
INPUT Exercise 4 : Skullcrusher (Dumbbell)
INPUT Exercise 5 : Incline Bench Press (Dumbbell)
1/1 [==============================] - 0s 7ms/step
Generated workout: ['Triceps Extension', 'Dumbbell Chest Press Together', 'Plate Push', 'Seated Palms Up Wrist Curl (Dumbbell)', 'Seated Palms Up Wrist Curl (Dumbbell)']
Exercise 1 : Triceps Extension
Exercise 2 : Dumbbell Chest Press Together
Exerci